# Image Processing for CZI Files

## Splitting the combined .czi file into separate channels

### List of Tasks

1. Read CZI file from a given folder.
2. Read metadata for each file and based on channel number split into separate channels.
3. Save each channel as a separate tif/tiff file with correct channel name without permission errors.

In [48]:
import glob
import os
import re
import shutil
import time
import cv2 as cv
import imageio
import nrrd
import numpy as np
import SimpleITK as sitk
import tifffile as tiff
from aicspylibczi import CziFile
from scipy import ndimage
from skimage import exposure, io
import matplotlib.pyplot as plt
from skimage import data

### Enter the exact/full path of the folder where images are located.

In [2]:
c_path = r"C:\Users\keshavgubbi\Desktop\LinesReg\v"
original_path = c_path + r"\original"

In [3]:
if not os.path.exists(original_path):
    print(f"Creating {original_path}")
    os.makedirs(original_path, exist_ok=True)

In [4]:
ref_ch_num = int(input("Enter Reference Channel Number:"))

Enter Reference Channel Number: 1


In [19]:
#start = time.time()
for file in os.listdir(c_path):
    if file.endswith(".czi"):
        print(file)
        name, ext = file.split(".")

        czi = CziFile(os.path.join(c_path, file))
        max_channels = range(*czi.dims_shape()[0]["C"])
        max_slices = range(*czi.dims_shape()[0]["Z"])
        
        # for ch_num in max_channels:
        for ch_num in range(len(max_channels)):
            print('Processing ch_num:', ch_num, '...........')
            ref_image_list = sig_image_list = []
            for z_plane in max_slices:
                imgarray, shp = czi.read_image(B=0, S=0, C=ch_num, T=0, Z=z_plane)
                print(f"Working with slice {z_plane}....")
                if ch_num == ref_ch_num:
                    ref_image_list.append(np.squeeze(imgarray))
                else:
                    sig_image_list.append(np.squeeze(imgarray))
        RImage = np.stack(ref_image_list).astype('uint8')
        SImage = np.stack(sig_image_list).astype('uint8')  
        
        print(len(RImage))
        print(len(SImage))
        print(f"Adding slice {z_plane} to {name}_ref.tif")
        with tiff.TiffWriter(os.path.join(original_path, f"{name}_ref.tif"), imagej=True) as tifw:
                tifw.write(RImage)
        print(f"Adding slice {z_plane} to {name}_sig.tif")
        with tiff.TiffWriter(os.path.join(original_path, f"{name}_sig.tif"), imagej=True) as tifw:
                tifw.write(SImage)

shhGFP_1_HuClyntagRFP.czi
Processing ch_num: 0 ...........
Working with slice 0....
Working with slice 1....
Working with slice 2....
Working with slice 3....
Working with slice 4....
Working with slice 5....
Working with slice 6....
Working with slice 7....
Working with slice 8....
Working with slice 9....
Working with slice 10....
Working with slice 11....
Working with slice 12....
Working with slice 13....
Working with slice 14....
Working with slice 15....
Working with slice 16....
Working with slice 17....
Working with slice 18....
Working with slice 19....
Working with slice 20....
Working with slice 21....
Working with slice 22....
Working with slice 23....
Working with slice 24....
Working with slice 25....
Working with slice 26....
Working with slice 27....
Working with slice 28....
Working with slice 29....
Working with slice 30....
Working with slice 31....
Working with slice 32....
Working with slice 33....
Working with slice 34....
Working with slice 35....
Working with sl

## Image Processing the tif files

In [23]:
line_name = input("Enter Line_name:")
tag_name = input("Enter tag name or name of the stain used:")

line_path = original_path
processed_path = line_path + "/processed/"
processed_for_average_path = line_path + "/processed_for_average/"

Enter Line_name: shhGFP
Enter tag name or name of the stain used: HUClyntag


In [24]:
def image_details(f):
    print("********Image Details**************")
    img = io.imread(f)
    print(type(img))
    print("Dtype:", img.dtype)
    print("Shape:", img.shape)
    return img


def convert_to_8bit(f):
    print("********Checking Image Type********")
    # img = io.imread(f)
    dtype = f.dtype
    print(dtype)
    if dtype != "uint8":
        f.astype("uint8", copy=False)
        print(dtype)
    return f.astype("uint8")


def read_voxel_size(f):
    # print("********Reading Voxel Size********")
    im = sitk.ReadImage(f)
    width, height = im.GetSpacing()
    # print('Current voxel size:', width, height)
    return width, height


def ce(f):
    # i = img_as_float(io.imread(f)).astype(np.float64)
    logarithmic_corrected = exposure.adjust_log(f, 3)
    return logarithmic_corrected


def _rotate(src, angle):
    # angle in degrees
    rotated_matrix = ndimage.rotate(src, angle=angle, reshape=False)
    return rotated_matrix


def tiff_unstackAndrestack(f):
    """
    :param f: tiff file
    :return: rotated_image_stack
    #1. Iterate through each file as a tiff file.
    #2. split into individual pages //Unstacking
    #3. rotate each page and save the rotated_page into a new list
    #4. restack each array from the list
    """
    with tiff.TiffFile(f, mode="r+b") as tif:
        print(f" Processing {tif} for rotation...")
        for page in tif.pages:
            rotated_page = _rotate(page.asarray(), theta)
            rotated_page_list.append(rotated_page)
            rotated_image_stack = np.stack(rotated_page_list)
    return rotated_image_stack.astype("uint8")

In [25]:
def split_and_rename(f):
    filename, exte = f.split(".")
    _first, _last = filename.split("_", 1)
    fishnum, tag = _last.split("_", 1)
    # print(_first, fishnum, ext)
    return _first, fishnum

In [38]:
%matplotlib notebook
import matplotlib.pyplot as plt

def visualize_stack(f):
    plt.imshow(f)
    plt.show()

In [40]:
#Contrast Enhancement Function
from skimage.filters.rank import enhance_contrast
def contrast_enhance(f):
    pass
    

In [62]:
alpha = 3.0 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)
def tiff_UnstackAndrestackForContrastEnhancement(f):
    contrast_enhanced_list = []
    with tiff.TiffFile(f, mode="r+b") as tif:
        print(f" Processing {tif} for Enhancing Contrast...")
        for page in tif.pages:
            contrast_enhanced_page = cv.convertScaleAbs(page.asarray(), alpha=alpha, beta=beta)
            contrast_enhanced_list.append(contrast_enhanced_page)
            contrast_enhanced_stack = np.stack(contrast_enhanced_list)
    return contrast_enhanced_stack.astype("uint8")

In [63]:
for item in os.listdir(line_path):
    if item.endswith(".tif"):

        # ****Contrast Enhancement, 8bit conversion, fixing voxel Depth******#
#         g = tiff.imread(os.path.join(line_path, item))
#         print(f"The file {item} has dimensions : {g.shape} and is of type: {g.dtype} ")
#         print(len(g))
#         CE_image = ce(g)
#         print(CE_image.shape)

        CE_image = tiff_UnstackAndrestackForContrastEnhancement(os.path.join(line_path, item))
        print(f"Creating file {item} ...")
        with tiff.TiffWriter(os.path.join(line_path, item), imagej=True) as tifw:
            tifw.write(CE_image.astype("uint8"),metadata={"spacing": 1.0, "unit": "um", "axes": "ZYX"})

#         #***********Rotation*********************************#
#         print(f'Image stack to be rotated: {item}')
#         theta = float(input('Enter the angle by which image to be rotated:'))
#         rotated_page_list = []
#         rotated_image = tiff_unstackAndrestack(os.path.join(line_path, item))
#         print(f'Creating Rotated Image: rotated_{item}')
#         with tiff.TiffWriter(os.path.join(line_path, f"{item}"), imagej=True) as tifw:
#             tifw.write(rotated_image.astype('uint8'), metadata={'spacing': 1.0, 'unit': 'um', 'axes': 'ZYX'})
#         print('*********************************************************')

In [ ]:
# **********Final Saving of Images to respective folders********
print("Final Saving of Images to respective folders!")
if not os.path.exists(processed_path):
    print(f"Creating {processed_path}")
    os.makedirs(processed_path, exist_ok=True)

if not os.path.exists(processed_for_average_path):
    print(f"Creating {processed_for_average_path}")
    os.makedirs(processed_for_average_path, exist_ok=True)

In [ ]:
for item in os.listdir(line_path):
    if item.endswith(".tif") and re.search("ref", str(item)):
        print(f"Image stack to be saved: {item}")
        # Read the data back from file
        readdata = tiff.imread(os.path.join(line_path, item))
        name, fn = split_and_rename(item)

        print(f"Creating {name}_{fn}_{tag_name}.nrrd")
        nrrd.write(
            os.path.join(processed_path, f"{name}_{fn}_{tag_name}.nrrd"),
            readdata,
            index_order="C",
        )

        print(f"Creating {name}_{fn}_{tag_name}.tif")
        with tiff.TiffWriter(
            os.path.join(processed_for_average_path, f"{name}_{fn}_{tag_name}.tif"),
            imagej=True,
        ) as tifw:
            tifw.write(
                readdata.astype("uint8"),
                metadata={"spacing": 1.0, "unit": "um", "axes": "ZYX"},
            )

In [ ]:
for item in os.listdir(line_path):
    if item.endswith(".tif") and re.search("sig", str(item)):
        # Read the data back from file
        readdata = tiff.imread(os.path.join(line_path, item))
        name = split_and_rename(item)

        print(f"Creating {name}_{fn}_GFP.nrrd")
        nrrd.write(
            os.path.join(processed_path, f"{name}_{fn}_GFP.nrrd"),
            readdata,
            index_order="C",
        )

        print(f"Creating {name}_{fn}_GFP.tif")
        with tiff.TiffWriter(
            os.path.join(processed_for_average_path, f"{name}_{fn}_GFP.tif"),
            imagej=True,
        ) as tifw:
            tifw.write(
                readdata.astype("uint8"),
                metadata={"spacing": 1.0, "unit": "um", "axes": "ZYX"},
            )

end = time.time()
print("total Execution Time:", end - start, "s")